# Load Dataframe from CSV and transform to PubAnnotation/Json format

## Auxiliary methods...

In [ ]:
def create_denotation(start, end, id_counter, tipo):
    span = {}
    span['begin'] = start
    span['end'] = end

    denotation = {}
    denotation['id'] = "T{}".format(id_counter)
    denotation['obj'] = tipo
    denotation['span'] = span
    
    return denotation

In [ ]:
def create_relation(id_counter, tipo_rel, subj_entity, obj_entity):
    relation = {}
    relation['id'] = "R{}".format(id_counter)
    relation['pred'] = tipo_rel
    relation['subj'] = subj_entity
    relation['obj'] = obj_entity
    
    return relation

In [ ]:
def get_all_denotations_from_sentence(df, sentence, entity_indexes):
    all_denotations = []
    
    denotation_id = 1
    
    offset = 0
    subsentence = sentence
    
    # dicionario com as entidades que pertencem a essa sentença
    # mapeia cada ID de entidade do dataset para o ID correspondente no json do kindred (i.e. T1, T2, T3...)
    # vai ser usado depois para verificar se as entidades das relações pertecem ou não a mesma sentença
    entity_id_to_kindred_id_dict = {}
    
    for entity_index in entity_indexes:
        entity = df.loc[(df.p_index == p_index) & (df.entity_index == entity_index), "entity"].values[0]
        
        entity_id = df.loc[(df.p_index == p_index) & (df.entity_index == entity_index), "entity_id"].values[0]
        
        tipo = df.loc[(df.p_index == p_index) & (df.entity_index == entity_index), "tipo_final"].values[0]

        # "limpa" a sentença das entidades que já foram processadas
        entity_start_index = subsentence.index(entity) + offset
        entity_end_index = entity_start_index + len(entity)
        offset = entity_end_index
        subsentence = subsentence[subsentence.index(entity) + len(entity):]

        denotation = create_denotation(
            entity_start_index,
            entity_end_index,
            denotation_id,
            tipo)
        
        denotation_id += 1
        
        all_denotations.append(denotation)
        
        # adiciona essa entidade no dicionario de entidades da sentença
        entity_id_to_kindred_id_dict[entity_id] = denotation['id']
        
    return all_denotations, entity_id_to_kindred_id_dict

In [ ]:
def get_denotations_from_doc(df, sentence, doc_index):
    all_denotations = []
    
    denotation_id = 1
    
    # dicionario que mapeia cada ID de entidade do dataset para o ID correspondente no json do kindred (i.e. T1, T2, T3...)
    entity_id_to_kindred_id_dict = {}
    
    offset = 0
    subsentence = sentence
    
    # lista com todas os IDs das entidades que pertencem a esse documento
    entity_ids = df.loc[df.doc_index == doc_index, "entity_id"].values
    
    for entity_id in entity_ids:
        entity = df.loc[df.entity_id == entity_id, "entity"].values[0]
        
        tipo = df.loc[df.entity_id == entity_id, "tipo_final"].values[0]

        # "limpa" a sentença das entidades que já foram processadas
        entity_start_index = subsentence.index(entity) + offset
        entity_end_index = entity_start_index + len(entity)
        offset = entity_end_index
        subsentence = subsentence[subsentence.index(entity) + len(entity):]

        denotation = create_denotation(
            entity_start_index,
            entity_end_index,
            denotation_id,
            tipo)
        
        denotation_id += 1
        
        all_denotations.append(denotation)
        
        # adiciona essa entidade no dicionario de entidades da sentença
        entity_id_to_kindred_id_dict[entity_id] = denotation['id']
        
    return all_denotations, entity_id_to_kindred_id_dict

In [ ]:
def get_all_relations_from_sentence(df, sentence, entity_indexes, entity_id_to_kindred_id_dict):
    all_relations = []
    
    relation_id = 1
    
    # se a relação tem uma entidade em uma sentença e outra entidade em outra sentença, essa relação vai ser descartada
    # o kindred só é capaz de reconhecer relações entre entidades em uma mesma sentença
    discarded_relations = []
    
    for entity_index in entity_indexes:
        entity = df.loc[(df.p_index == p_index) & (df.entity_index == entity_index), "entity"].values[0]
        
        entity_id = df.loc[(df.p_index == p_index) & (df.entity_index == entity_index), "entity_id"].values[0]
        
        # corels é uma String com as "entidades objetos" de cada relação, separadas por um espaço
        # EX: H2-dftre765-12 H2-dftre765-9 H2-dftre765-1
        corels = df.loc[(df.p_index == p_index) & (df.entity_index == entity_index), "corel"].values[0]
        
        # para ignorar os casos em que não tem nenhuma relação, fazer esse if x==x:
        if (corels == corels):
            # tiporels é uma lista com as relações
            # EX: [autor_de, natural_de, participante_em]
            tiporels = df.loc[(df.p_index == p_index) & (df.entity_index == entity_index), "tiporel"].values[0].split()
            
            # transforma o corels em uma lista e processa elemento a elemento (cada elemento "corel" é o id do obj da relação)
            for corel_index, corel in enumerate(corels.split()):
                
                # só processar a relação se o obj está na mesma sentença que o subj, ou seja, está no dict
                # a parte do "corel_index < len(tiporels)" é porque tem uns dados bugados no dataset
                if corel in entity_id_to_kindred_id_dict and corel_index < len(tiporels):
                    
                    relation = create_relation(
                        relation_id,
                        tiporels[corel_index],
                        entity_id_to_kindred_id_dict[entity_id],
                        entity_id_to_kindred_id_dict[corel])
                    
                    relation_id += 1
                    
                    all_relations.append(relation)
                   
                # se não vai utilizar a relação, então adiciona ela na lista de descarte
                # a parte do "len(obj_entity) == 1" também é por causa de um bug no dataset
                else:
                    obj_entity = df.loc[df.entity_id == corel, "entity"].values
                    if (len(obj_entity) == 1):
                        text = "the subj {} ({}) is in sentence {}, but the obj {} ({}) is in other sentence".format(entity, entity_id, p_index, obj_entity[0], corel)
                        discarded_relations.append(text)
                        
    return all_relations, discarded_relations

In [ ]:
def get_relations_from_doc(df, sentence, doc_index, entity_id_to_kindred_id_dict):
    all_relations = []
    
    relation_id = 1
    
    # lista com todas os IDs das entidades que pertencem a esse documento
    entity_ids = df.loc[df.doc_index == doc_index, "entity_id"].values
    
    for entity_id in entity_ids:
        entity = df.loc[df.entity_id == entity_id, "entity"].values[0]
        
        # corels é uma String com as "entidades objetos" de cada relação, separadas por um espaço
        # EX: H2-dftre765-12 H2-dftre765-9 H2-dftre765-1
        corels = df.loc[df.entity_id == entity_id, "corel"].values[0]
        
        # para ignorar os casos em que não tem nenhuma relação, fazer esse if x==x:
        if (corels == corels):
            # tiporels é uma lista com as relações
            # EX: [autor_de, natural_de, participante_em]
            tiporels = df.loc[df.entity_id == entity_id, "tiporel"].values[0].split()
            
            # transforma o corels em uma lista e processa elemento a elemento (cada elemento "corel" é o id do obj da relação)
            for corel_index, corel in enumerate(corels.split()):
                
                # só processar a relação se o obj está no dict
                # a parte do "corel_index < len(tiporels)" é porque tem uns dados bugados no dataset
                if corel in entity_id_to_kindred_id_dict and corel_index < len(tiporels):
                    
                    relation = create_relation(
                        relation_id,
                        tiporels[corel_index],
                        entity_id_to_kindred_id_dict[entity_id],
                        entity_id_to_kindred_id_dict[corel])
                    
                    relation_id += 1
                    
                    all_relations.append(relation)
                        
    return all_relations

## First Approach: create a Json file for each sentence

In [ ]:
import pandas as pd
import json

df = pd.read_csv("dataset_processed_6.csv")

all_discarded_relations = []

# p_index ranges from 1 to 2273 (total number of sentences)
p_indexes_set = set(df["p_index"])

for p_index in p_indexes_set:
    
    sentence = df.loc[df.p_index == p_index, "p_sentence_processed"].values[0]
    
    # lista com a quantidade de entidades que tem na sentença
    entity_indexes = df.loc[df.p_index == p_index, "entity_index"].values
    
    all_denotations, entity_id_to_kindred_id_dict = get_all_denotations_from_sentence(df,
                                                                                      sentence,
                                                                                      entity_indexes)
    
    all_relations, discarded_relations = get_all_relations_from_sentence(df, 
                                                                         sentence,
                                                                         entity_indexes,
                                                                         entity_id_to_kindred_id_dict)
    
    all_discarded_relations.append(discarded_relations)
        
    # cada arquivo json vai ser "sentença + entidades + relações"
    json_per_sentence = {}     
    json_per_sentence['text'] = sentence
    json_per_sentence['denotations'] = all_denotations
    json_per_sentence['relations'] = all_relations
    
    with open("kindred/sentence_p_index_{}.json".format(p_index), "w+", encoding="utf-8") as fileWriter:
        json.dump(json_per_sentence, fileWriter, indent=2, ensure_ascii=False)

        
# salva em um arquivo todas as relações que foram descartadas
with open("kindred/discarded_relations.txt", "w+", encoding="utf-8") as fileWriter:
    for discarded_relations in all_discarded_relations:
        for text in discarded_relations:
            print(text, file=fileWriter)
            
print("DONE")

## Second Approach: create a Json file for each document (so we can use all relations)

In [ ]:
import pandas as pd
import json

df = pd.read_csv("dataset_processed_6.csv")

all_discarded_relations = []

# doc_index ranges from 0 to 128 (total number of documents)
doc_indexes_set = set(df["doc_index"])

for doc_index in sorted(doc_indexes_set):
    
    big_sentence = ""
    
    # total number of sentences in document
    p_indexes_set = set(df.loc[df.doc_index == doc_index, "p_index"].values)
    
    # build a big sentence with all sentences from document
    for p_index in sorted(p_indexes_set):
        sentence = df.loc[df.p_index == p_index, "p_sentence_processed"].values[0]

        if (sentence.endswith('.')):
            big_sentence = big_sentence + " " + sentence
        else:
            big_sentence = big_sentence + " " + sentence + "."
    
    print(sorted(p_indexes_set))
    #print(big_sentence)
    print("\n")
    all_denotations, ids_dict = get_denotations_from_doc(df, big_sentence, doc_index)

    all_relations = get_relations_from_doc(df, big_sentence, doc_index, ids_dict)

    big_json = {}
    big_json['text'] = big_sentence
    big_json['denotations'] = all_denotations
    big_json['relations'] = all_relations


    with open("kindred/big_sentence_doc_{}.json".format(doc_index), "w+", encoding="utf-8") as fileWriter:
        json.dump(big_json, fileWriter, indent=2, ensure_ascii=False)
        
print("DONE")

## Third Approach: create a Json file for each relation (with big sentences)

In [ ]:
import pandas as pd
import json

relation_counter = 0

df = pd.read_csv("dataset_processed_6.csv")

# doc_index ranges from 0 to 128 (total number of documents)
doc_indexes_set = set(df["doc_index"])

for doc_index in sorted(doc_indexes_set):
    
    big_sentence = ""
    
    # total number of sentences in document
    p_indexes_set = set(df.loc[df.doc_index == doc_index, "p_index"].values)
    
    # build a big sentence with all sentences from document
    for p_index in sorted(p_indexes_set):
        sentence = df.loc[df.p_index == p_index, "p_sentence_processed"].values[0]

        if (sentence.endswith('.')):
            big_sentence = big_sentence + " " + sentence
        else:
            big_sentence = big_sentence + " " + sentence + "."
    
    all_denotations, entity_id_to_kindred_id_dict = get_denotations_from_doc(df, big_sentence, doc_index)

    all_relations = get_relations_from_doc(df, big_sentence, doc_index, entity_id_to_kindred_id_dict)

    # aqui nós já temos todas as entidades e relações do documento...
    # pegar cada uma das relações e criar um json que só tenha 1 relação e 2 entidades
    for relation in all_relations:
        # pega o id TX das entidades sujeito e objeto
        subj_kindred_id = relation['subj']
        obj_kindred_id = relation['obj']

        # pega as denotations entidades sujeito e objeto
        for denotation in all_denotations:
            if (denotation['id'] == subj_kindred_id):
                subj_denotation = denotation.copy()
            elif (denotation['id'] == obj_kindred_id):
                obj_denotation = denotation.copy()

        # substitui os IDs para começarem do 1
        relation['id'] = "R1"
        relation['subj'] = "T1"
        relation['obj'] = "T2"
        subj_denotation['id'] = "T1"
        obj_denotation['id'] = "T2"

        big_json = {}
        big_json['text'] = big_sentence
        big_json['denotations'] = [subj_denotation, obj_denotation]
        big_json['relations'] = [relation]
        
        with open("teste/big_sentence_doc_{}.json".format(relation_counter), "w+", encoding="utf-8") as fileWriter:
            json.dump(big_json, fileWriter, indent=2, ensure_ascii=False)
            
        relation_counter += 1
                                   
print("DONE")

## EXTRA STEP:
Salvar os arquivos no Drive e executar o Kindred_Experiments pelo Google Colab.

Depois de pegar os arquivos que foram gerados pelo modelo, pode seguir pro próximo passo...

## RELATION EXTRACTION WITH KINDRED
A execução do relation_by_relation acabou estourando a memória do Google Colab, então tive que executar localmente.

In [ ]:
!pip install kindred
!python -m spacy download pt_core_news_md

In [ ]:
import kindred
import time
import sys

default_stdout = sys.stdout

fold_count = 10

# o load estava causando alguns problemas de codificação pelo Jupyter (pelo Colab não dava problema)
# eu abri o código interno da lib kindred e alterei uma parte do código que era "open(filename)" por "open(filename, utf-8)"
mainCorpus = kindred.load("pubannotation", "kindred_relation_by_relation")

i = 0

if True:
    for trainCorpus, goldCorpus in mainCorpus.nfold_split(fold_count):

        sys.stdout = open("results_big_fold_{}.txt".format(i), "w+")

        start_time = time.time()

        classifier = kindred.RelationClassifier(model="pt_core_news_md")

        classifier.train(trainCorpus)

        testCorpus = goldCorpus.clone()

        testCorpus.removeRelations()

        classifier.predict(testCorpus)

        kindred.evaluate(goldCorpus, testCorpus, metric='all', display=True)

        end_time = time.time()

        print("--- {} seconds ---".format(end_time - start_time))

        i += 1

        sys.stdout.close()

    sys.stdout = default_stdout

## Separate Stratified Folds 
Isso não é mais utilizado.
Esse código foi usado para separar os folds quando o dataset inteiro foi centralizado em uma única sentença. No entanto, o Kindred acertou 0 TPs nesse caso, então optei por fazer de outra maneira.

In [ ]:
if False:
    from sklearn.model_selection import StratifiedKFold
    import json

    ids = [relation["id"] for relation in all_relations] # na verdade isso nem precisa, na doc disse que só as labels já servem

    tiporels = [relation["pred"] for relation in all_relations]

    skf = StratifiedKFold(n_splits=10)

    i = 0

    for train_indexes, test_indexes in skf.split(ids, tiporels):

        train_fold = [all_relations[index] for index in train_indexes]

        test_fold = [all_relations[index] for index in test_indexes]

        train_json = {}
        train_json['text'] = big_sentence
        train_json['denotations'] = all_denotations
        train_json['relations'] = train_fold

        test_json = {}
        test_json['text'] = big_sentence
        test_json['denotations'] = all_denotations
        test_json['relations'] = test_fold

        with open("kindred_folds/train_fold_{}.json".format(i), "w+", encoding="utf-8") as fileWriter:
            json.dump(train_json, fileWriter, indent=2, ensure_ascii=False)

        with open("kindred_folds/test_fold_{}.json".format(i), "w+", encoding="utf-8") as fileWriter:
            json.dump(test_json, fileWriter, indent=2, ensure_ascii=False)

        i += 1

    print("DONE")

## CALCULAR METRICAS MICRO E MACRO A PARTIR DOS RESULTADOS

In [1]:
for fold in range(10):

    with open("dbpedia/metrics{}.txt".format(fold), "r", encoding="utf-8") as fileReader:
        i = 0
        dicP = []
        dicR = []
        dicF = []

        for line in fileReader:
            if(line.partition("----")[1]):
                break
            else:
                texto1 = line.partition("\tP:")[2]
                precision = texto1.partition(" R:")[0]
                texto2 = texto1.partition(" R:")[2]
                recall = texto2.partition(" F1:")[0]
                fscore = texto2.partition(" F1:")[2]

                if (False):
                    print(line)
                    print("P = {}".format(precision))
                    print("R = {}".format(recall))
                    print("F = {}".format(fscore))

                dicP.append(float(precision))
                dicR.append(float(recall))
                dicF.append(float(fscore))

                i += 1
        
        print(fold)
        textP = "P = {}".format(sum(dicP)/len(dicP))
        print(textP)
        textR = "R = {}".format(sum(dicR)/len(dicR))
        print(textR)
        textF = "F = {}".format(sum(dicF)/len(dicF))
        print(textF)
        print("\n")

0
P = 0.6811679
R = 0.6843985
F = 0.669432


1
P = 0.665911
R = 0.6149571999999999
F = 0.6169362


2
P = 0.6664977999999999
R = 0.6337047
F = 0.6368026


3
P = 0.6876503999999999
R = 0.5913721
F = 0.6044445


4
P = 0.6533356000000001
R = 0.6265207999999999
F = 0.6222332999999999


5
P = 0.6170353000000001
R = 0.5927911
F = 0.5915109000000001


6
P = 0.7122315000000001
R = 0.6621200999999999
F = 0.6643102999999999


7
P = 0.7066344
R = 0.6118888
F = 0.6237676000000001


8
P = 0.7071309
R = 0.6307379
F = 0.6418549


9
P = 0.6399335
R = 0.5899413000000001
F = 0.6015599


